In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from models.GTS.DCRNN import DCRNN

from torch_geometric.utils import to_dense_adj, dense_to_sparse
from torch_geometric_temporal.dataset import METRLADatasetLoader, PemsBayDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split, StaticGraphTemporalSignalBatch

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from glob import glob
import yaml
from easydict import EasyDict as edict

from collections import defaultdict


In [3]:
config_file = glob('./config/GTS/METR-LA/1.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [4]:
metr_la = np.load('./data/METR-LA/node_values.npy')
adj_matrix = np.load('./data/METR-LA/adj_mat.npy')

loader = METRLADatasetLoader(raw_data_dir='./data/METR-LA')

In [5]:
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=3)

In [6]:
train_dataset, dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [7]:
validation_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.5)

In [8]:
class DecoderModel(nn.Module):
    def __init__(self, config):
        super(DecoderModel, self).__init__()
        self.output_dim = config.dataset.pred_step
        self.hidden_dim = config.hidden_dim

        self.decoder_dcrnn = DCRNN(config)
        self.prediction_layer = nn.Linear(self.hidden_dim, self.output_dim)

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.fill_(0.1)

    def forward(self, inputs, adj, hidden_state, weight_matrix=None):
        decoder_hidden_state = hidden_state

        decoder_hidden_state = self.decoder_dcrnn(inputs, adj, hidden_state=decoder_hidden_state, weight_matrix=weight_matrix)
        prediction = self.prediction_layer(decoder_hidden_state[-1].view(-1, self.hidden_dim))

        output = prediction.view(inputs.shape[0], self.output_dim)

        return output, decoder_hidden_state


class GTS_Forecasting_Module(nn.Module):
    def __init__(self, config):
        super(GTS_Forecasting_Module, self).__init__()

        self.config = config

        self.nodes_num = config.nodes_num

        self.nodes_feas = config.node_features

        self.encoder_step = config.encoder_step
        self.decoder_step = config.decoder_step

        self.encoder_model = DCRNN(config)
        self.decoder_model = DecoderModel(config)


    def forward(self, inputs, targets, adj_matrix, weight_matrix=None):
        # DCRNN encoder
        encoder_hidden_state = None
        for i in range(self.encoder_step):
            encoder_hidden_state = self.encoder_model(inputs[:,i].unsqueeze(dim=-1), adj_matrix,
                                                      encoder_hidden_state, weight_matrix)

        # DCRNN decoder
        outputs = []
        decoder_input = torch.zeros((self.nodes_num, 1)).to('cuda:3')
        
        decoder_hidden_state = encoder_hidden_state
        for j in range(self.decoder_step):
            output, decoder_hidden_state = self.decoder_model(decoder_input, adj_matrix, decoder_hidden_state, weight_matrix)
            outputs.append(output)
            
            decoder_input = targets[:,j].unsqueeze(dim=-1)

        outputs = torch.cat(outputs, dim=-1)
        return outputs


In [9]:
model = GTS_Forecasting_Module(config)

In [10]:
model = model.to('cuda:3')

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fnc = nn.L1Loss().to('cuda:3')
model.train()

GTS_Forecasting_Module(
  (encoder_model): DCRNN(
    (recurrent): ModuleList(
      (0): DCRNN(
        (conv_x_z): DConv()
        (conv_x_r): DConv()
        (conv_x_h): DConv()
      )
    )
  )
  (decoder_model): DecoderModel(
    (decoder_dcrnn): DCRNN(
      (recurrent): ModuleList(
        (0): DCRNN(
          (conv_x_z): DConv()
          (conv_x_r): DConv()
          (conv_x_h): DConv()
        )
      )
    )
    (prediction_layer): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [18]:
results = defaultdict(list)
best_val_loss = np.inf

for epoch in range(3):
    model.train()
    
    train_loss = []
    for batch in train_dataset:
        input_x = batch.x[:,0,:].to('cuda:3')
        target = batch.y.to('cuda:3')
    
        y_hat = model(input_x, adj_matrix = batch.edge_index.to('cuda:3'), targets=target)
        loss = loss_fnc(y_hat, target)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        train_loss += [float(loss.data.cpu().numpy())]
        
    train_loss = np.stack(train_loss).mean()
    results['train_loss'] += [train_loss]
    
    
    model.eval()
    
    val_loss = []
    for batch in validation_dataset:
        input_x = batch.x[:,0,:].to('cuda:3')
        target = batch.y.to('cuda:3')
        
        y_hat = model(input_x, adj_matrix = batch.edge_index.to('cuda:3'), targets=target)
        loss = loss_fnc(y_hat, target)
        val_loss += [float(loss.data.cpu().numpy())]
        
    val_loss = np.stack(val_loss).mean()
    results['val_loss'] += [val_loss]
    
    
    if type(adj_matrix) == dict:
        results['val_adj_matirix'] += adj_matrix
    else:
        results['val_adj_matirix'] += [adj_matrix.detach().cpu()]

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        
        

AttributeError: 'numpy.ndarray' object has no attribute 'detach'